# Analysis of using FD to compute minimum plan length

This notebook uses the following packages:
- [Pandas](https://pandas.pydata.org/)
- [Seaborn](https://seaborn.pydata.org/) 
- [Numpy](https://numpy.org/)
- [Statsmodels](https://www.statsmodels.org/stable/index.html)

The notebook uses the csv files generated by the output processing script from the benchmarking folder. This notebook analysis the solvers for each scenario.

In [3]:
# import the required libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [73]:
# set the path to the csv file
csv_interim_file = "~/Work/Data/FondASP/interim.csv"
csv_final_file = "~/Work/Data/FondASP/final.csv"
csv_weak_plan_file =  "~/Work/Data/FondASP/weak_plan.csv"

## Comparison of controller states and weak plan length

In [74]:
df_weak_plans = pd.read_csv(csv_weak_plan_file)
df_weak_plans = df_weak_plans.set_index(["scenario", "instance"])
df_weak_plans["min_states"] = df_weak_plans["weak_plan"] + 1
df_weak_plans.head()

weak_plan   fd_time  min_states
scenario        instance                                 
spiky-tireworld p09               8  0.004960           9
                p01               8  0.002528           9
                p05               8  0.003649           9
                p07               8  0.004436           9
                p11               8  0.005785           9

In [84]:
df_solver = pd.read_csv(csv_final_file)
df_solver = df_solver.set_index(["scenario", "instance"])
df_solver.head()

solver    total-time  ground-time  \
scenario           instance                                               
triangle-tireworld p05       fondsat-glucose  18658.434277   234.273223   
                   p02       fondsat-glucose      8.835798     2.739566   
                   p03       fondsat-glucose    257.171451    17.322236   
                   p04       fondsat-glucose   2190.970903    57.145693   
                   p01       fondsat-glucose      0.449260     0.216553   

                               solve-time  states  
scenario           instance                        
triangle-tireworld p05       18334.031495      40  
                   p02           5.537110      16  
                   p03         235.854187      24  
                   p04        2114.116259      32  
                   p01           0.098212       8

In [85]:
df_states = pd.merge(df_solver,df_weak_plans,how='inner',on=['scenario','instance'])
df_states["required_steps"] = df_states["states"] - df_states["min_states"]
df_states.head()

solver    total-time  ground-time  \
scenario           instance                                               
triangle-tireworld p05       fondsat-glucose  18658.434277   234.273223   
                   p02       fondsat-glucose      8.835798     2.739566   
                   p03       fondsat-glucose    257.171451    17.322236   
                   p04       fondsat-glucose   2190.970903    57.145693   
                   p01       fondsat-glucose      0.449260     0.216553   

                               solve-time  states  weak_plan   fd_time  \
scenario           instance                                              
triangle-tireworld p05       18334.031495      40         10  0.002815   
                   p02           5.537110      16          4  0.001726   
                   p03         235.854187      24          6  0.001965   
                   p04        2114.116259      32          8  0.002290   
                   p01           0.098212       8          2  0.001412   

                             min_states  required_steps  
scenario           instance                              
triangle-tireworld p05               11              29  
                   p02                5              11  
                   p03                7              17  
                   p04                9              23  
                   p01                3               5

In [86]:
# compute the difference
df_states["backbone_size"] = df_states["min_states"]/df_states["states"]
df_states

solver    total-time  ground-time  \
scenario           instance                                               
triangle-tireworld p05       fondsat-glucose  18658.434277   234.273223   
                   p02       fondsat-glucose      8.835798     2.739566   
                   p03       fondsat-glucose    257.171451    17.322236   
                   p04       fondsat-glucose   2190.970903    57.145693   
                   p01       fondsat-glucose      0.449260     0.216553   
...                                      ...           ...          ...   
miner              p07       fondsat-minisat    215.669369    75.421949   
                   p07             asp-opt-1    597.880000   163.110000   
                   p38       fondsat-glucose    859.956553   510.185038   
                   p38       fondsat-minisat   1217.307959   558.391352   
                   p38             asp-opt-1  14360.469000   652.799000   

                               solve-time  states  weak_plan   fd_time  \
scenario           instance                                              
triangle-tireworld p05       18334.031495      40         10  0.002815   
                   p02           5.537110      16          4  0.001726   
                   p03         235.854187      24          6  0.001965   
                   p04        2114.116259      32          8  0.002290   
                   p01           0.098212       8          2  0.001412   
...                                   ...     ...        ...       ...   
miner              p07         121.915774      20          5  0.005466   
                   p07         434.770000      20          5  0.005466   
                   p38         147.663444      22          8  0.019979   
                   p38         427.909291      22          8  0.019979   
                   p38       13707.670000      22          8  0.019979   

                             min_states  required_steps   benefit  
scenario           instance                                        
triangle-tireworld p05               11              29  0.275000  
                   p02                5              11  0.312500  
                   p03                7              17  0.291667  
                   p04                9              23  0.281250  
                   p01                3               5  0.375000  
...                                 ...             ...       ...  
miner              p07                6              14  0.300000  
                   p07                6              14  0.300000  
                   p38                9              13  0.409091  
                   p38                9              13  0.409091  
                   p38                9              13  0.409091  

[1079 rows x 10 columns]

In [87]:
# any instance where weak plan is larger than controller size
df_states.query(f"min_states > states")

solver    total-time  ground-time  \
scenario               instance                                               
blocksworld-ipc08      p04       fondsat-glucose     36.897760    18.181389   
                       p07       fondsat-glucose     29.467396    14.311139   
first-responders-ipc08 p17       fondsat-glucose    204.418671     9.852455   
elevators              p03       fondsat-glucose      6.227139     2.731568   
                       p06       fondsat-glucose     62.224409    12.711397   
                       p08       fondsat-glucose  55922.003443    71.973709   
                       p09       fondsat-glucose  31808.334206    62.160282   
                       p07       fondsat-glucose     68.635396    12.047927   
zenotravel             p05       fondsat-glucose    355.225006   246.202420   
                       p03       fondsat-glucose    250.268731   139.580459   
                       p04       fondsat-glucose    217.870466   124.267035   

                                   solve-time  states  weak_plan   fd_time  \
scenario               instance                                              
blocksworld-ipc08      p04          17.111176      14         14  0.004373   
                       p07          13.825884      13         14  0.004441   
first-responders-ipc08 p17         191.682253      17         17  0.002764   
elevators              p03           2.651401      16         17  0.002005   
                       p06          44.858309      23         28  0.002792   
                       p08       55806.850072      36         43  0.004259   
                       p09       31703.290082      36         36  0.004649   
                       p07          52.048535      23         31  0.003010   
zenotravel             p05          74.307009      15         20  0.011789   
                       p03          92.401037      17         18  0.011062   
                       p04          73.914558      13         13  0.009558   

                                 min_states  required_steps   benefit  
scenario               instance                                        
blocksworld-ipc08      p04               15              -1  1.071429  
                       p07               15              -2  1.153846  
first-responders-ipc08 p17               18              -1  1.058824  
elevators              p03               18              -2  1.125000  
                       p06               29              -6  1.260870  
                       p08               44              -8  1.222222  
                       p09               37              -1  1.027778  
                       p07               32              -9  1.391304  
zenotravel             p05               21              -6  1.400000  
                       p03               19              -2  1.117647  
                       p04               14              -1  1.076923

In [91]:
df_solver.query(f"solver=='fondsat-glucose' and scenario=='blocksworld-ipc08' and instance=='p04'")

,,solver,total-time,ground-time,solve-time,states
scenario,instance,,,,,
blocksworld-ipc08,p04,fondsat-glucose,36.89776,18.181389,17.111176,14


## Include the data from interim steps

In [68]:
df_steps = pd.read_csv(csv_interim_file, index_col=['scenario', 'instance'])
df_steps = df_steps.drop(["solver"], axis=1)
df_steps.head()

total-time  ground-time  solve-time  iteration
scenario           instance                                                
triangle-tireworld p05         0.206952     0.196070    0.010882          1
                   p05         0.340053     0.298641    0.041412          2
                   p05         0.514824     0.436142    0.078682          3
                   p05         0.543848     0.387321    0.156527          4
                   p05         0.726407     0.636349    0.090058          5

In [69]:

df_all = pd.merge(df_states, df_steps,how='inner',on=['scenario', 'instance'])
df_all.head()

solver  total-time_x  ground-time_x  \
scenario          instance                                           
blocksworld-ipc08 p01       asp-opt-1           4.8           3.92   
                  p01       asp-opt-1           4.8           3.92   
                  p01       asp-opt-1           4.8           3.92   
                  p01       asp-opt-1           4.8           3.92   
                  p01       asp-opt-1           4.8           3.92   

                            solve-time_x  states  weak_plan   fd_time  \
scenario          instance                                              
blocksworld-ipc08 p01               0.88       9          6  0.007733   
                  p01               0.88       9          6  0.007733   
                  p01               0.88       9          6  0.007733   
                  p01               0.88       9          6  0.007733   
                  p01               0.88       9          6  0.007733   

                            min_states  required_steps   benefit  \
scenario          instance                                         
blocksworld-ipc08 p01                7               2  0.777778   
                  p01                7               2  0.777778   
                  p01                7               2  0.777778   
                  p01                7               2  0.777778   
                  p01                7               2  0.777778   

                            total-time_y  ground-time_y  solve-time_y  \
scenario          instance                                              
blocksworld-ipc08 p01           0.262677       0.240141      0.022536   
                  p01           0.450134       0.375291      0.074842   
                  p01           0.660318       0.528942      0.131376   
                  p01           1.079477       0.860872      0.218605   
                  p01           1.293451       0.945255      0.348196   

                            iteration  
scenario          instance             
blocksworld-ipc08 p01               1  
                  p01               2  
                  p01               3  
                  p01               4  
                  p01               5

In [70]:
df_all.query(f"scenario=='islands' and instance=='p01'")

solver  total-time_x  ground-time_x  solve-time_x  \
scenario instance                                                         
islands  p01       asp-opt-1         0.032          0.032           0.0   
         p01       asp-opt-1         0.032          0.032           0.0   
         p01       asp-opt-1         0.032          0.032           0.0   
         p01       asp-opt-1         0.032          0.032           0.0   
         p01       asp-opt-1         0.032          0.032           0.0   
         p01       asp-opt-1         0.032          0.032           0.0   
         p01       asp-opt-1         0.032          0.032           0.0   
         p01       asp-opt-1         0.032          0.032           0.0   
         p01       asp-opt-1         0.032          0.032           0.0   
         p01       asp-opt-1         0.032          0.032           0.0   

                   states  weak_plan   fd_time  min_states  required_steps  \
scenario instance                                                            
islands  p01            4          1  0.001278           2               2   
         p01            4          1  0.001278           2               2   
         p01            4          1  0.001278           2               2   
         p01            4          1  0.001278           2               2   
         p01            4          1  0.001278           2               2   
         p01            4          1  0.001278           2               2   
         p01            4          1  0.001278           2               2   
         p01            4          1  0.001278           2               2   
         p01            4          1  0.001278           2               2   
         p01            4          1  0.001278           2               2   

                   benefit  total-time_y  ground-time_y  solve-time_y  \
scenario instance                                                       
islands  p01           0.5      0.016544       0.009843      0.006701   
         p01           0.5      0.026366       0.018794      0.007572   
         p01           0.5      0.015000       0.015000      0.000000   
         p01           0.5      0.016000       0.016000      0.000000   
         p01           0.5      0.026745       0.013989      0.012757   
         p01           0.5      0.039097       0.021888      0.017209   
         p01           0.5      0.013000       0.013000      0.000000   
         p01           0.5      0.019000       0.019000      0.000000   
         p01           0.5      0.019000       0.019000      0.000000   
         p01           0.5      0.021000       0.021000      0.000000   

                   iteration  
scenario instance             
islands  p01               1  
         p01               2  
         p01               1  
         p01               2  
         p01               1  
         p01               2  
         p01               1  
         p01               2  
         p01               1  
         p01               2